In [3]:
%pip install python-dotenv

In [4]:
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv

In [ ]:
# Load biến môi trường từ file .env
load_dotenv()

# Lấy giá trị biến API_KEY
openai.api_key = os.getenv("API_KEY")

print("API Key:", openai.api_key)

In [6]:
client = OpenAI(api_key=openai.api_key)

In [ ]:
import pandas as pd
# from google.colab import files

In [ ]:
# uploaded = files.upload()  # Tải lên file Excel

In [ ]:
# file_name = list(uploaded.keys())[0]
file_name = './demo_data.xlsx'
df = pd.read_excel(file_name)
df.head()

In [ ]:
## 4. Tạo Prompt từ bảng dữ liệu
def generate_prompt_from_df(df):
    table_str = df.to_string(index=False)
    prompt = f"""
    You are a financial analyst. Below is a table of daily financial data:

    {table_str}

    Please:
    1. Generate a **professional financial summary report** based on this data according to Vietnamese accounting standard Circular 200/2014/TT-BTC.
    2. Include key sections in the report: Revenue, Costs, Profit, and meaningful comments.
    3. Additionally, output a **clean and concise Excel-style table** (with headers) containing only the **main financial indicators**, such as:
      - Total Revenue
      - Total Cost of Goods Sold
      - Total Operating Expenses
      - Total Net Profit

    The output should contain two parts:
    - A human-readable report summary
    - A clearly formatted financial summary table, ready to export to Excel

    Use professional English.
    """
    return prompt

In [ ]:
prompt = generate_prompt_from_df(df)
print(prompt)


    You are a financial analyst. Below is a table of daily financial data:

    Posting Date        Account Name                Description  Amount Department Code Project Code Related Party
  2025-01-01                Cash  Customer payment received    5000             D01         P001    Customer A
  2025-01-02 Accounts Receivable Invoice issued to customer    7000             D01         P001    Customer B
  2025-01-02                Cash        Payment to supplier   -3000             D02         P002    Supplier X
  2025-01-03                Cash   Petty cash replenishment   -1500             D03         P003    Employee T
  2025-01-03 Accounts Receivable         Invoice correction   -1000             D01         P001    Customer B

    Please:
    1. Generate a **professional financial summary report** based on this data according to Vietnamese accounting standard Circular 200/2014/TT-BTC.
    2. Include key sections in the report: Revenue, Costs, Profit, and meaningful comments.

In [ ]:
## 5. Gọi GPT để sinh báo cáo
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a financial analyst"},
        {"role": "user", "content": prompt}
    ]
)

report_text = response.choices[0].message.content
print(report_text)

### Professional Financial Summary Report

#### Report Date: January 2025

#### Overview
This report provides a financial summary for the period covering January 1 to January 3, 2025, prepared in accordance with the Vietnamese accounting standard Circular 200/2014/TT-BTC. The focus is on revenue, costs, and profit analysis, providing insights into the financial performance during this period.

#### Financial Summary

**Revenue:**
- Total revenue generated from the issuance of invoices to customers amounted to VND 7,000. However, there was a subsequent invoice correction of VND 1,000. 
- Adjusted total revenue stands at VND 6,000.

**Costs:**
- The company incurred a payment of VND 3,000 to a supplier, which is considered as cost of goods sold.
- Operating expenses include a petty cash replenishment of VND 1,500.

**Profit Evaluation:**
- **Net Revenue:** VND 6,000
- **Cost of Goods Sold (COGS):** VND 3,000
- **Operating Expenses:** VND 1,500
- **Net Profit:** VND 1,500

#### Comments
-

In [ ]:
# 7. Trích bảng từ output GPT (đơn giản)
import re
lines = report_text.splitlines()
table_lines = [l for l in lines if "|" in l and "---" not in l]

# Bỏ dấu | đầu cuối rồi mới tách theo dấu |
rows = [re.split(r"\s*\|\s*", l.strip().strip('|')) for l in table_lines]

if len(rows) > 1:
    df_summary = pd.DataFrame(rows[1:], columns=rows[0])
    df_summary.to_excel("financial_summary.xlsx", index=False)
    files.download("financial_summary.xlsx")
else:
    print("⚠️ No summary table found.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
prompt = f'How do you calculate Total Cost of Goods Sold from {df}'
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a financial analyst"},
        {"role": "user", "content": prompt}
    ]
)

report_text = response.choices[0].message.content
print(report_text)

To calculate the Total Cost of Goods Sold (COGS) from the provided data, we need to identify and sum up all the costs directly associated with the production and delivery of goods sold by a company. These costs typically include raw materials, components, labor, and manufacturing overhead incurred in making the product.

In the dataset you provided, you'll need to determine which transactions represent these types of costs. From the sample data you provided, it seems like transaction IDs and their descriptions vary but generally don't fully describe typical COGS transactions directly. However, we can make some assumptions:

1. **Payment to suppliers**: This typically forms part of COGS if it is related to purchasing materials or components needed for production. The transaction on "2025-01-02" with the description "Payment to supplier" for the amount "-3000" likely relates to COGS because it involves a supplier payment.

2. Other transactions: Cash transactions and accounts receivable 

1. Exact Match Accuracy (EMA) <br>
Compares the entire extracted table from GPT with the ground truth. If every cell matches exactly, the prediction is considered 100% correct.
```python

accuracy = (df_output.equals(df_ground_truth))

2. Cell-level Accuracy / Match Rate <br>
Compares each individual cell in the predicted table against the corresponding cell in the reference table.
```python

total_cells = df_output.size
matched_cells = (df_output.values == df_ground_truth.values).sum()
cell_accuracy = matched_cells / total_cells

3. Levenshtein Similarity / Fuzzy Matching
Uses fuzzy string matching (via fuzzywuzzy or rapidfuzz) to allow small variations in values, such as punctuation or formatting differences.
```python

from rapidfuzz import fuzz
fuzz.ratio("6,000", "6000")  # ~95% similarity
